In [ ]:
pip install py-aiohanspell
#비교할게 있어야함  konlpy 라이브러리 Hannanum

In [ ]:
pip install pygwalker

In [1]:
#전처리
import konlpy
from konlpy.tag import Twitter
# from konlpy.tag import Hannanum
from konlpy.tag import Okt
#맞춤법
from aiohanspell import spell_checker
#시각화
import pygwalker as pyg

In [2]:
import time
from openpyxl import Workbook
import pandas as pd
from bs4 import BeautifulSoup
#
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#chrome driver 링크 연결 없이 바로 하는 법
#%pip install webdriver-manager (install받아야함)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
#문자열에서 특수문자 제거
import re
#엑셀문서
import openpyxl
#경고메세지 지워줌
import warnings 
warnings.filterwarnings('ignore')

In [3]:
#엑셀 시트 생성
wb = Workbook(write_only=True)
ws = wb.create_sheet()


In [7]:
#크롬드라이버 링크 연결
# path='C:\chromedriver_win32\chromedriver.exe'
# driver= webdriver.Chrome(path) ->
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options())

#스크롤 시 풀 스크린으로 켜지게되면 우측에 '추천영상' 때문에 댓글보다 더 스크롤이 내려가게 됨
#하프 사이즈의 스크린으로 켜게 되면 추천영상이 일정이상 새로고침되지 않음 
driver.set_window_size(800, 1100)
#이 상태에서는 링크창에 data;,라고 적힌채로 대기상태에 들어감

In [8]:
#1. 검색어 입력받기
search=input()
driver.implicitly_wait(3)


#2. 유튜브 실행
driver.get("https://www.youtube.com")
#3초안에 웹페이지를 로딩하면 바로넘어가거나, 3초를 기다림
driver.implicitly_wait(3)

#get(링크)로 안하고 이것만 실행하면
#현재 페이지에 합쳐진다!!!

######검색#######


#3. 검색어 send
#ID,CLASS_NAME으로 하면 send가 안되거나 click조차 안될때가 있었음
#NAME으로 하면 둘 다 해결됨 이유는 모름?
search_text=driver.find_element(By.NAME,'search_query')

#로딩이 다 안됐기때문에 1초 기다려줌
driver.implicitly_wait(1)
time.sleep(1)

#4. 검색어 전송
search_text.send_keys(search+" -Shorts")

driver.implicitly_wait(5)

#검색버튼 누르기
# search_btn=driver.find_element(By.CLASS_NAME,'style-scope.ytd-searchbox')
# search_btn.click()
# driver.implicitly_wait(3)

#5. 검색(엔터 누르기)
search_text.send_keys(Keys.ENTER)

In [ ]:
######영상 선정######
id='dismissible'



In [25]:
######스크롤하기######
#댓글 전체 스크랩하기 위해

#275개의 댓글 >> 23.5s , 23.2s , 24.0s 23~24s


#화면 최하단으로 스크롤 이동 : scrollTo(0, document.body.scrollHeight)
#자바 스크립트 문법 >execute_script(자바스크립트를 실행) > scrollTo

#window.scrollTo(x-좌표, y-좌표)
#시작위치지정(댓글창이 시작되는 위치)
#화면사이즈를 반으로 줄이고나니 스크롤을 끝까지 내린 후에 댓글이 생김
driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
driver.implicitly_wait(10)

#스크롤 이전 높이
#화면 바깥으로 삐져나간 부분까지 포함해서 전체 글의 길이를 scrollHeight
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    #스크롤의 y좌표를 가장아래(scrollHeight)까지 내림
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    #스크롤 후 높이 구하기
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    #끝까지 스크롤 한 뒤 멈추기
    if new_height == last_height:
        break
    last_height = new_height

driver.implicitly_wait(1.5)

#댓글 약 3000개 1m 41.6s

######대댓글 토글 열기######
replies = driver.find_elements(By.CLASS_NAME,"more-button")
driver.implicitly_wait(1)

for reply in replies:
    #click도 안됨 send_keys도 안됨
    driver.execute_script("arguments[0].click();", reply)
    time.sleep(1.5) 

In [ ]:
# ######유튜브 팝업 닫기######

# #find_element_by_css_selector는 과거의 속성 find_element를 쓰는게 나음
# try:
#     driver.find_element(By.CSS_SELECTOR,"#dismiss-button > a").click()
# except:
#     pass

In [49]:
# ######대댓글 토글 열기######
# replies = driver.find_elements(By.CLASS_NAME,"more-button")
# driver.implicitly_wait(1)

# for reply in replies:
#     #click도 안됨 send_keys도 안됨
#     driver.execute_script("arguments[0].click();", reply)
#     time.sleep(1.5) 

# # reply.click()
# # reply.send_keys(Keys.ENTER)
# # 이거 안됨 스크립트로 해결함 근데 11분 18초 걸림!!!!!
# # 2m 49s
# # 댓글 약 3000개 
# # 댓글 309개 7m 2.7s

In [26]:
######댓글 가져오기#######
#page_source
#브라우저에 보이는 그대로의 HTML, 크롬 개발자 도구의 Element 탭 내용과 동일.
html_source = driver.page_source
#BeautifulSoup(html문서,'html.parser')
soup = BeautifulSoup(html_source, 'html.parser')

comment_list = soup.select("yt-formatted-string#content-text")

comment_final = []

twt = Twitter()
okt = Okt()
stop_words=set(['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한'])

for i in range(len(comment_list)):

    temp_comment = comment_list[i].text 
    
    # 맞춤법 고치기 
    #주간 두간 안바뀌도록 replace
    temp_comment = temp_comment.replace(search,"\""+ search +"\"")
    temp_comment = await spell_checker.check(temp_comment) #char[] hi = ['a','a','a']
    temp_comment = temp_comment.checked  

    
    #replace는 str() 함수기 때문에 -> 밑에서 특수문자 제거해 줄 것이라 안해도 됨
    # temp_comment = str(temp_comment).replace("\""+ search +"\"",search)
    #list to str 인데 str()은 ㄹㅇ ""붙이는거고 얘는 바꿔주는거 ['a','b','c'] ','.join이라 a,b,c해줌
    

    temp_comment = str(twt.normalize(temp_comment))
    temp_comment = temp_comment.split(" ")
    
    #str상태

    temp_comment = list(filter(lambda n : n.find('@')!=0,temp_comment))
    temp_comment = list(filter(lambda n : n.find('http')==-1,temp_comment))
    
    
    temp_comment = list(map(lambda n : re.sub(r"[^\uAC00-\uD7A30-9a-zA-Zぁ-ゔァ-ヴー々〆〤一-龥\s]","",n).replace(u'\xa0',u''),temp_comment))
    

    # temp_comment = " ".join(temp_comment)
    # temp_comment = okt.morphs(temp_comment, stem= True)

    #5글자 이상이면 맞춤법 교정 한번 더 하기    

    temp_comment = list(filter(None,temp_comment))
    comment_final.append(temp_comment) #append안에 list형식임



comment_final = sum(comment_final,[])
        
comment_final = list(filter(lambda n : len(n)!=1,comment_final))

print(comment_final)
# le = len(comment_final)
# print(le)

comment_dict={}

for key in comment_final:
    if key not in comment_dict:
        #True 반환
        # print('미포함')
        comment_dict[key] = 1

    else:
        #False 반환
        # print('포함')
        comment_dict[key] = comment_dict[key]+1
    
sorted_1 = sorted(comment_dict.items(), key = lambda item: item[1], reverse = True)
comment_dict = dict(sorted_1)

print(comment_dict)



['눈웃음에', '짓다가', '두간님', '샤우팅에', '견들', '크게', '뜨시는', '넘고', '커엽자너', '어어', '광주넘모', '영광이었습니다', '처음부터', '예고도', '없이', '그렇게', '예쁜', '눈으로', '쳐다보시면', '심장에', '무리', '갑니다', '아침은', '항상', '자희', '님의', '눈웃음', '때문에', '출근하는데', '기운이', '업되네요', '노래', '너무', '잘하시네요', '노래도', '잘하시고', '외모도', '천상의', '미모를', '뽐내시는', '우리', '짜지', '님과', '번만이라도', '핥아보고', '싶은', '성대를', '가지신', '우리', '두간님의', '현실', '합방이라니', '누나', '주거욧', '엔티티님', '여기', '올라갑니다', '두간님', '진짜', '실력이', '난리', '났어', '대박이야', '눈웃음에', '녹고', '갑니다', '129', '이걸', '라이브로', '부르는', '사람', '있다니', '눈웃음', '너무', '예쁘시고', '이번', '영상', '호강에', '호강까지', '너무', '행복했습니다', '128', '라이브로', '부르는데', '이렇게', '부른다고', '오름', '진짜', '눈웃음', '대박이네요', '쩐다', '자희', '님의', '청아한', '음색', '두간님의', '파워풀한', '음색', '너무', '좋아요', '눈웃음', '정말', '매력적', '목소리도', '눈웃음도', '외출복도', '정말', '님이랑', '어울려요', '진짜', '최애', '조합이잖아요', '두간님', '군다', '하기', '전에', '합방', '많이', '해주세요', '128', '147', 'unravel440', '완전', '감각542', '611', '700', '내가', '죽으려고1003', 'rain갓두간', '두간님', '노래', '부르시네', '고음도', '내시고', '짜지', '님은', '눈웃음이', '너무', '좋아', '근데', '진짜'

In [27]:
######추출한 아이디 댓글 저장하기######

import csv

# youtube_pd = pd.DataFrame(comment_dict)
# # youtube_pd = youtube_pd.transpose()
# youtube_pd.to_excel('result.csv')


with open('dogan1.csv','w',encoding='utf-8') as f:
    w = csv.writer(f)
    w.writerow(comment_dict.keys())
    w.writerow(comment_dict.values())


In [ ]:
# #엑셀파일 자동으로 열라고했는데 실패한 흔적들
# #엑셀 파일 가져오기
# open=openpyxl.load_workbook('result.xlsx')
# #시트명 취득 
# sheet=open.get_sheet_names('Sheet1')
# sheet.cell(row=1, column=1).value

In [ ]:
#aiohanspell
result = await spell_checker.check(u'생리미친자식왜약먹어도 안낫지') 
# print(result.as_dict())  # dict로 출력
print(result)
print(result.original)
print(result.checked)
#Checked(result=True, original='안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.', 
# checked='안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.', errors=4, words=OrderedDict([('안녕하세요.', 2), 
# ('저는', 0), ('한국인입니다.', 2), ('이', 2), ('문장은', 2), ('한글로', 0), ('작성됐습니다.', 1)]), time=0.10472893714904785)

#request 로 naver에 요청해서 받는 것 같아;;???허걱스